In [30]:
BU=input('''Selecciona la BU: BEAUTY JEW ACC CALZADO ROPA ''') 

#def CreateProyeccionActual(BU):

div =   {'BEAUTY': "[Division Code] = 'BEAUTY'", 
            'JEW': "[Division Code] IN ('W-JEW', 'M-JEW')",
            'ACC': "[Division Code] IN ('W-ACC', 'M-ACC')",
        'CALZADO':"[Division Code] in ('W-SHO','M-SHO')",
           'ROPA':"[Division Code] in ('W-CLO','M-CLO')"} 



import datetime
import pandas as pd
import numpy as np
import tensorflow as tf
import numpy as np
#import matplotlib.pyplot as plt
import os 
import get_data as gd

today = datetime.date.today()
carpeta_input = R'S:\BI\3. MERCHANDISING\FRAMEWORK\FRAMEWORK\ZIMA\BuffersBasicosV1\input'
carpeta_output = R'S:\BI\3. MERCHANDISING\FRAMEWORK\FRAMEWORK\ZIMA\BuffersBasicosV1\output'           

end_date = str(today)

# vale, la proyeccion es con base en el año anterior entonces, se necesita un query grande, en este caso se toma desde la primera semana del año anterior                                    
df_inv_ventas_hist = gd.get_inv_venta_hist(BU, div, '2022-01-09', end_date, carpeta_input, carpeta_output, 33)

#T117 2112513083
print("Detectando semanas faltantes... ") #-------------------------------------------------------------------------------------------------
# Aquí se crea un rango de fechas para cada SKU y tienda

# Creando un dataframe con los datos faltantes
datos_faltantes = pd.DataFrame(columns=df_inv_ventas_hist.columns)
numero_datos_faltantes = pd.DataFrame(columns=['Location Code', 'SKU', 'Cantidad'])

SKUS = df_inv_ventas_hist['SKU'].unique()
for sku in SKUS: 
    df = df_inv_ventas_hist[df_inv_ventas_hist['SKU'] == sku] # for
    tiendas = df['Location Code'].unique()
    for tienda in tiendas:
        df_tienda = df[df['Location Code'] == tienda]

        first_week = df_tienda["Date"].min().strftime("%Y-%m-%d")
        last_week = df_tienda["Date"].max().strftime("%Y-%m-%d")

        # Se crea un rango de fechas con el fin de identificar si hay datos faltantes
        weeks = pd.date_range(start=first_week, end=last_week, freq='W')

        # Se crea un dataframe con el rango de fechas
        df_weeks = pd.DataFrame(weeks, columns=['Date'])

        df_tienda = pd.merge(df_weeks, df_tienda, how='left', on='Date')

        # Se rellenan los datos faltantes con nan
        df_tienda = df_tienda.fillna(np.nan)

        # Se rellenan los vacios con interpolacion lineal * deje ID con nans para identificarlos
        
        df_tienda['Inventario'] = df_tienda['Inventario'].interpolate()
        df_tienda['Ventas'] = df_tienda['Ventas'].interpolate()

        df_tienda['Location Code'].fillna(tienda, inplace=True)
        df_tienda['SKU'].fillna(sku, inplace=True)

        Cantidad = df_tienda['ID'].isna().sum()
        faltantes = pd.DataFrame({'Location Code': tienda, 'SKU': sku, 'Cantidad': Cantidad}, index=[0])
        numero_datos_faltantes = pd.concat([numero_datos_faltantes, faltantes])

        # Se agrega en el dataframe de datos faltantes
        datos_faltantes = pd.concat([datos_faltantes, df_tienda[df_tienda['ID'].isna()]])

        # tiempo interpolacion 1 min 40 seg
        # 54 seg
        # 48 seg

df_inv_ventas_hist['Date'] = pd.to_datetime(df_inv_ventas_hist['Date'])
df_inv_ventas_hist = pd.concat([df_inv_ventas_hist, datos_faltantes]).reset_index(drop=True)
df_inv_ventas_hist = df_inv_ventas_hist.sort_values(by=['SKU', 'Location Code', 'Date']).reset_index(drop=True) ##

print('Porcentaje de datos interpolados:', str((df_inv_ventas_hist['ID'].isnull().sum() / len(df_inv_ventas_hist))*100) + '%')

def windowed_dataset(series, window_size, batch_size, shuffle_buffer):
    
    dataset = tf.data.Dataset.from_tensor_slices(series)

    dataset = dataset.window(window_size + 1, shift=1, drop_remainder=True)
    # Single batch
    dataset = dataset.flat_map(lambda window: window.batch(window_size + 1))

    dataset = dataset.map(lambda window: (window[:-1], window[-1]))

    dataset = dataset.shuffle(shuffle_buffer)
    
    dataset = dataset.batch(batch_size).prefetch(1)
    
    return dataset

from datetime import datetime


Consultando inventarios... 
Consultando ventas... 
Consultando clones... 
Preparando data... 
Identificando características de tiendas y productos....
Detectando semanas faltantes... 
Porcentaje de datos interpolados: 1.259714906626395%


In [1]:
data = df_inv_ventas_hist[['SKU', 'Location Code', 'Ventas', 'Inventario', 'Date']]
data['Location Code'].astype('category')

NameError: name 'df_inv_ventas_hist' is not defined

In [45]:
# encode Location Code
data['Location Code'] =  data['Location Code'].astype('category')
data['Location Code'] = data['Location Code'].cat.codes

C:\Users\fcolin\AppData\Local\Temp\ipykernel_20204\712706890.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['Location Code'] =  data['Location Code'].astype('category')
C:\Users\fcolin\AppData\Local\Temp\ipykernel_20204\712706890.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['Location Code'] = data['Location Code'].cat.codes


In [76]:
# Econde date
data.loc[0,'Date']
# transform timestamp to datetime
data['Date'] = pd.to_datetime(data['Date'])
# encode 

data['Date'] = data['Date'].astype('category')
data['Date'] = data['Date'].cat.codes

data['Date'].unique()


C:\Users\fcolin\AppData\Local\Temp\ipykernel_20204\1299829324.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['Date'] = pd.to_datetime(data['Date'])
C:\Users\fcolin\AppData\Local\Temp\ipykernel_20204\1299829324.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['Date'] = data['Date'].astype('category')
C:\Users\fcolin\AppData\Local\Temp\ipykernel_20204\1299829324.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_in

array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16,
       17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33,
       34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50,
       51, 52, 53, 54, 55, 56], dtype=int8)

In [22]:
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt

def plot_series(time, series, format="-", start=0, end=None):
    """Helper function to plot our time series"""
    plt.plot(time[start:end], series[start:end], format)
    plt.xlabel("Time")
    plt.ylabel("Value")
    plt.grid(False)

def trend(time, slope=0):
    """Define the trend through slope and time"""
    return slope * time

def seasonal_pattern(season_time):
    """Arbitrary definition of a seasonality pattern"""
    return np.where(season_time < 0.1,
                    np.cos(season_time * 6 * np.pi),
                    2 / np.exp(9 * season_time))

def seasonality(time, period, amplitude=1, phase=0):
    """Repeats a pattern at each period"""
    season_time = ((time + phase) % period) / period
    return amplitude * seasonal_pattern(season_time)

def noise(time, noise_level=1, seed=None):
    """Adds white noise to the series"""
    rnd = np.random.RandomState(seed)
    return rnd.randn(len(time)) * noise_level

def generate_time_series():
    # Temporal dimension: 4 years of data
    time = np.arange(4 * 365 + 1, dtype="float32")

    # The initial series is nothing more than a straight line which we will then modify with the other functions
    y_intercept = 10
    slope = 0.005
    series = trend(time, slope) + y_intercept

    # Add seasonality
    amplitude = 50
    series += seasonality(time, period=365, amplitude=amplitude)

    # Add noise
    noise_level = 3
    series += noise(time, noise_level, seed=51)
    
    return time, series


# Let's save the parameters of our time series in the dataclass
#@dataclass
class G:
    TIME, SERIES = generate_time_series()
    SPLIT_TIME = 1100 # on day 1100 the training period will end. The rest will belong to the validation set
    WINDOW_SIZE = 20 # how many data points will we take into account to make our prediction
    BATCH_SIZE = 32 # how many items will we supply per batch
    SHUFFLE_BUFFER_SIZE = 1000 # we need this parameter to define the Tensorflow sample buffer
    

In [23]:


def train_val_split(time, series, time_step=G.SPLIT_TIME):
	"""Divide the time series into training and validation set"""
	time_train = time[:time_step]
	series_train = series[:time_step]
	time_valid = time[time_step:]
	series_valid = series[time_step:]

	return time_train, series_train, time_valid, series_valid

def windowed_dataset(series, window_size=G.WINDOW_SIZE, batch_size=G.BATCH_SIZE, shuffle_buffer=G.SHUFFLE_BUFFER_SIZE):
	"""
	We create time windows to create X and y features.
	For example, if we choose a window of 30, we will create a dataset formed by 30 points as X
	"""
	dataset = tf.data.Dataset.from_tensor_slices(series)
	dataset = dataset.window(window_size + 1, shift=1, drop_remainder=True)
	dataset = dataset.flat_map(lambda window: window.batch(window_size + 1))
	dataset = dataset.shuffle(shuffle_buffer)
	dataset = dataset.map(lambda window: (window[:-1], window[-1]))
	dataset = dataset.batch(batch_size).prefetch(1)
	return dataset

In [25]:
dataset

<WindowDataset element_spec=DatasetSpec(TensorSpec(shape=(), dtype=tf.int64, name=None), TensorShape([]))>

In [29]:
dataset = tf.data.Dataset.range(10)
dataset = dataset.window(5, shift=1, drop_remainder=True)

'''for window_dataset in dataset:
    for val in window_dataset:
        print(val.numpy(), end=" ")
    print()'''

# let's create the dataset with time windows
dataset = windowed_dataset(G.SERIES, G.WINDOW_SIZE, G.BATCH_SIZE, G.SHUFFLE_BUFFER_SIZE)

# we divide into training and validation set
train_val_split(G.TIME, G.SERIES)

(array([0.000e+00, 1.000e+00, 2.000e+00, ..., 1.097e+03, 1.098e+03,
        1.099e+03], dtype=float32),
 array([59.12849 , 60.274727, 63.495922, ..., 65.180084, 62.56855 ,
        63.612823], dtype=float32),
 array([1100., 1101., 1102., 1103., 1104., 1105., 1106., 1107., 1108.,
        1109., 1110., 1111., 1112., 1113., 1114., 1115., 1116., 1117.,
        1118., 1119., 1120., 1121., 1122., 1123., 1124., 1125., 1126.,
        1127., 1128., 1129., 1130., 1131., 1132., 1133., 1134., 1135.,
        1136., 1137., 1138., 1139., 1140., 1141., 1142., 1143., 1144.,
        1145., 1146., 1147., 1148., 1149., 1150., 1151., 1152., 1153.,
        1154., 1155., 1156., 1157., 1158., 1159., 1160., 1161., 1162.,
        1163., 1164., 1165., 1166., 1167., 1168., 1169., 1170., 1171.,
        1172., 1173., 1174., 1175., 1176., 1177., 1178., 1179., 1180.,
        1181., 1182., 1183., 1184., 1185., 1186., 1187., 1188., 1189.,
        1190., 1191., 1192., 1193., 1194., 1195., 1196., 1197., 1198.,
        119

In [ ]:
print(producto, tienda)
        
df = df_inv_ventas_hist[(df_inv_ventas_hist['SKU'] == producto) & (df_inv_ventas_hist['Location Code'] == tienda)].reset_index(drop=True)
df = df[['Date', 'Ventas']]
df['Ventas']= df['Ventas'].astype(int)

# ordenando por fecha
df = df.sort_values(by=['Date']).reset_index(drop=True)
df['time'] = range(len(df))

time = df['time'].values
series = df['Ventas'].values

# Define the split time
split_time = int(round(len(df)*0.80,0))

# Get the train set 
time_train = time[:split_time]
x_train = series[:split_time]

# Get the validation set
time_valid = time[split_time:]
x_valid = series[split_time:]    

window_size = 4
batch_size = 30
shuffle_buffer_size = 1000

dataset = windowed_dataset(x_train, window_size, batch_size, shuffle_buffer_size)    

l0 = tf.keras.layers.Dense(1, input_shape=[window_size])

model = tf.keras.models.Sequential([
tf.keras.layers.Conv1D(filters=64, kernel_size=3,
                    strides=1, padding="causal",
                    activation="relu",
                    input_shape=[window_size, 1]),
tf.keras.layers.LSTM(64, return_sequences=True),
tf.keras.layers.LSTM(64),
tf.keras.layers.Dense(1),
tf.keras.layers.Lambda(lambda x: x * 400)
])

#model.compile(loss="mse", optimizer=tf.keras.optimizers.SGD(learning_rate=1e-6, momentum=0.9), metrics=["accuracy"])
model.compile(loss=tf.keras.losses.Huber(), optimizer=tf.keras.optimizers.SGD(learning_rate=1e-6, momentum=0.9), metrics=["mae"])

history = model.fit(dataset,epochs=100)

# Todo esto es para hacer el forecast en el tiempo de validacion
'''forecast = []
for time in range(len(series) - window_size):
    forecast.append(model.predict(series[time:time + window_size][np.newaxis]))

forecast = forecast[split_time - window_size:]
results = np.array(forecast).squeeze()

#Convert to a numpy array and drop single dimensional axes
results = np.array(forecast).squeeze()
results = np.round(abs(results), 0)

# Overlay the results with the validation set
time = df['time'].values
series = df['Ventas'].values
forecast = forecast[split_time - window_size:]
# plot_series(time_valid, (x_valid, abs(results)))

#print("Real: ", sum(x_valid[-7:]))
#print("Proyectada: ", sum(abs(results[-7:])))

#print(tf.keras.metrics.mean_squared_error(x_valid, results).numpy())
#print(tf.keras.metrics.mean_absolute_error(x_valid, results).numpy())'''
